# Ibis for dplyr Users

[R](https://www.r-project.org/) users familiar with [dplyr](https://dplyr.tidyverse.org/) are likely to find some parts of Ibis familiar.
In fact, some Ibis verbs have been named to match their corresponding dplyr verbs.

However, due to differences between Python and R and the design and goals of Ibis itself, those familiar with dplyr may notice some big between the two right away:

- **No pipe:** R's handy [magrittr pipe](https://magrittr.tidyverse.org/) (`%>%`) or native pipe (`|>`) don't exist in Python so you instead have to chain sequences of methods together with a period (`.`)
- **No unquoted column names:**: Non-standard evaluation is common in R but not present in Python. To reference column names in Ibis, you can use strings, properties (`t.my_col`), the `_` helper (e.g., `_.my_col`), or expressions like `s.contains`
- Wrapping complex expressions in parens to make the evaluate correctly
- (need to check but can group_by be used for other things that aggregates?. For instance:
    ```r
    starwars |>
      filter(!is.na(height)) |>
      group_by(species) |> 
      slice_max(height, n = 3)
   ```

   - ibis has more similar to dplyr+dbplyr (TODO: expand on this)


## Comparing Ibis and dplyr

Using the same example data and similar operations as in [Introduction to dplyr](https://dplyr.tidyverse.org/articles/dplyr.html), below you will find some examples of the more common dplyr operations and their Ibis equivalents.

### Loading Ibis

In [6]:
import ibis
import ibis.examples as ex
import ibis.selectors as s
from ibis import _
ibis.options.interactive = True

ModuleNotFoundError: No module named 'ibis'

### Loading example data

In R, datasets are typically lazily loaded with packages. For instance, the `starwars` dataset is packaged with dplyr, but is not loaded in memory before you start using it. Ibis provides many datasets in the `examples` module. So to be able to use the `starwars` dataset, you can use:

In [ ]:
starwars = ex.starwars.fetch()

### Inspecting the dataset with `head()`

Just like in R, you can use `head()` to inspect the beginning of a dataset. You can also specify the number of rows you want to get back by using the parameter `n` (default `n = 5`).

In R:

```r
head(starwars) # or starwars |> head()
```

With Ibis:

In [ ]:
starwars.head(6)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┓
┃ name           ┃ height ┃ mass    ┃ hair_color  ┃ skin_color  ┃ eye_color ┃ birth_year ┃ sex    ┃ gender    ┃ homeworld ┃ species ┃ films  ┃ vehicles ┃ starships ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━┩
│ string         │ int64  │ float64 │ string      │ string      │ string    │ float64    │ string │ string    │ string    │ string  │ string │ string   │ string    │
├────────────────┼────────┼─────────┼─────────────┼─────────────┼───────────┼────────────┼────────┼───────────┼───────────┼─────────┼────────┼──────────┼───────────┤
│ Luke Skywalker │    172 │    77.0 │ blond       │ fair        │ blue      │       19.0 │ male   │ masculine │ Tatooine  │ Human   │ NULL   │ NULL     │ NULL      │
│ C-3PO          │    167 │    75.0 │ NULL        │ gold        │ yellow    │      112.0 │ none   │ masculine │ Tatooine  │ Droid   │ NULL   │ NULL     │ NULL      │
│ R2-D2          │     96 │    32.0 │ NULL        │ white, blue │ red       │       33.0 │ none   │ masculine │ Naboo     │ Droid   │ NULL   │ NULL     │ NULL      │
│ Darth Vader    │    202 │   136.0 │ none        │ white       │ yellow    │       41.9 │ male   │ masculine │ Tatooine  │ Human   │ NULL   │ NULL     │ NULL      │
│ Leia Organa    │    150 │    49.0 │ brown       │ light       │ brown     │       19.0 │ female │ feminine  │ Alderaan  │ Human   │ NULL   │ NULL     │ NULL      │
│ Owen Lars      │    178 │   120.0 │ brown, grey │ light       │ blue      │       52.0 │ male   │ masculine │ Tatooine  │ Human   │ NULL   │ NULL     │ NULL      │
└────────────────┴────────┴─────────┴─────────────┴─────────────┴───────────┴────────────┴────────┴───────────┴───────────┴─────────┴────────┴──────────┴───────────┘

There is no `tail()` in Ibis because most databases do not support this operation.

Another method you can use to limit the number of rows returned by a query is `limit()` which also takes the `n` parameter.

In [ ]:
starwars.limit(3)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┓
┃ name           ┃ height ┃ mass    ┃ hair_color ┃ skin_color  ┃ eye_color ┃ birth_year ┃ sex    ┃ gender    ┃ homeworld ┃ species ┃ films  ┃ vehicles ┃ starships ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━┩
│ string         │ int64  │ float64 │ string     │ string      │ string    │ float64    │ string │ string    │ string    │ string  │ string │ string   │ string    │
├────────────────┼────────┼─────────┼────────────┼─────────────┼───────────┼────────────┼────────┼───────────┼───────────┼─────────┼────────┼──────────┼───────────┤
│ Luke Skywalker │    172 │    77.0 │ blond      │ fair        │ blue      │       19.0 │ male   │ masculine │ Tatooine  │ Human   │ NULL   │ NULL     │ NULL      │
│ C-3PO          │    167 │    75.0 │ NULL       │ gold        │ yellow    │      112.0 │ none   │ masculine │ Tatooine  │ Droid   │ NULL   │ NULL     │ NULL      │
│ R2-D2          │     96 │    32.0 │ NULL       │ white, blue │ red       │       33.0 │ none   │ masculine │ Naboo     │ Droid   │ NULL   │ NULL     │ NULL      │
└────────────────┴────────┴─────────┴────────────┴─────────────┴───────────┴────────────┴────────┴───────────┴───────────┴─────────┴────────┴──────────┴───────────┘

### Filtering rows with filter()

Ibis, like dplyr, has the `filter` method to select rows based on conditions.

With dplyr:

```r
starwars |>
  filter(skin_color == "light")
```

In Ibis:

In [ ]:
starwars.filter(_.skin_color == "light")

┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┓
┃ name               ┃ height ┃ mass    ┃ hair_color  ┃ skin_color ┃ eye_color ┃ birth_year ┃ sex    ┃ gender    ┃ homeworld ┃ species ┃ films  ┃ vehicles ┃ starships ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━┩
│ string             │ int64  │ float64 │ string      │ string     │ string    │ float64    │ string │ string    │ string    │ string  │ string │ string   │ string    │
├────────────────────┼────────┼─────────┼─────────────┼────────────┼───────────┼────────────┼────────┼───────────┼───────────┼─────────┼────────┼──────────┼───────────┤
│ Leia Organa        │    150 │    49.0 │ brown       │ light      │ brown     │       19.0 │ female │ feminine  │ Alderaan  │ Human   │ NULL   │ NULL     │ NULL      │
│ Owen Lars          │    178 │   120.0 │ brown, grey │ light      │ blue      │       52.0 │ male   │ masculine │ Tatooine  │ Human   │ NULL   │ NULL     │ NULL      │
│ Beru Whitesun lars │    165 │    75.0 │ brown       │ light      │ blue      │       47.0 │ female │ feminine  │ Tatooine  │ Human   │ NULL   │ NULL     │ NULL      │
│ Biggs Darklighter  │    183 │    84.0 │ black       │ light      │ brown     │       24.0 │ male   │ masculine │ Tatooine  │ Human   │ NULL   │ NULL     │ NULL      │
│ Lobot              │    175 │    79.0 │ none        │ light      │ blue      │       37.0 │ male   │ masculine │ Bespin    │ Human   │ NULL   │ NULL     │ NULL      │
│ Cordé              │    157 │     nan │ brown       │ light      │ brown     │        nan │ female │ feminine  │ Naboo     │ Human   │ NULL   │ NULL     │ NULL      │
│ Dormé              │    165 │     nan │ brown       │ light      │ brown     │        nan │ female │ feminine  │ Naboo     │ Human   │ NULL   │ NULL     │ NULL      │
│ Raymus Antilles    │    188 │    79.0 │ brown       │ light      │ brown     │        nan │ male   │ masculine │ Alderaan  │ Human   │ NULL   │ NULL     │ NULL      │
│ Rey                │   NULL │     nan │ brown       │ light      │ hazel     │        nan │ female │ feminine  │ NULL      │ Human   │ NULL   │ NULL     │ NULL      │
│ Poe Dameron        │   NULL │     nan │ brown       │ light      │ brown     │        nan │ male   │ masculine │ NULL      │ Human   │ NULL   │ NULL     │ NULL      │
│ …                  │      … │       … │ …           │ …          │ …         │          … │ …      │ …         │ …         │ …       │ …      │ …        │ …         │
└────────────────────┴────────┴─────────┴─────────────┴────────────┴───────────┴────────────┴────────┴───────────┴───────────┴─────────┴────────┴──────────┴───────────┘

In dplyr, you can specify multiple conditions separated with `,` that are then combined with the `&` operator:

```r
starwars |>
  filter(skin_color == "light", eye_color == "brown")
```

In Ibis, you can do the same by putting multiple conditions in a list:

In [ ]:
starwars.filter([_.skin_color == "light", _.eye_color == "brown"])

┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┓
┃ name              ┃ height ┃ mass    ┃ hair_color ┃ skin_color ┃ eye_color ┃ birth_year ┃ sex    ┃ gender    ┃ homeworld ┃ species ┃ films  ┃ vehicles ┃ starships ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━┩
│ string            │ int64  │ float64 │ string     │ string     │ string    │ float64    │ string │ string    │ string    │ string  │ string │ string   │ string    │
├───────────────────┼────────┼─────────┼────────────┼────────────┼───────────┼────────────┼────────┼───────────┼───────────┼─────────┼────────┼──────────┼───────────┤
│ Leia Organa       │    150 │    49.0 │ brown      │ light      │ brown     │       19.0 │ female │ feminine  │ Alderaan  │ Human   │ NULL   │ NULL     │ NULL      │
│ Biggs Darklighter │    183 │    84.0 │ black      │ light      │ brown     │       24.0 │ male   │ masculine │ Tatooine  │ Human   │ NULL   │ NULL     │ NULL      │
│ Cordé             │    157 │     nan │ brown      │ light      │ brown     │        nan │ female │ feminine  │ Naboo     │ Human   │ NULL   │ NULL     │ NULL      │
│ Dormé             │    165 │     nan │ brown      │ light      │ brown     │        nan │ female │ feminine  │ Naboo     │ Human   │ NULL   │ NULL     │ NULL      │
│ Raymus Antilles   │    188 │    79.0 │ brown      │ light      │ brown     │        nan │ male   │ masculine │ Alderaan  │ Human   │ NULL   │ NULL     │ NULL      │
│ Poe Dameron       │   NULL │     nan │ brown      │ light      │ brown     │        nan │ male   │ masculine │ NULL      │ Human   │ NULL   │ NULL     │ NULL      │
│ Padmé Amidala     │    165 │    45.0 │ brown      │ light      │ brown     │       46.0 │ female │ feminine  │ Naboo     │ Human   │ NULL   │ NULL     │ NULL      │
└───────────────────┴────────┴─────────┴────────────┴────────────┴───────────┴────────────┴────────┴───────────┴───────────┴─────────┴────────┴──────────┴───────────┘

If you want to combine multiple conditions, in dplyr, you could do:

```r
starwars |>
  filter(
      (skin_color == "light" & eye_color == "brown") |
       species == "Droid"
  )
```

In Ibis, this would be:

In [ ]:
starwars.filter(
    ((_.skin_color == "light") & (_.eye_color == "brown")) |
    (_.species == "Droid")
)

┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┓
┃ name              ┃ height ┃ mass    ┃ hair_color ┃ skin_color  ┃ eye_color ┃ birth_year ┃ sex    ┃ gender    ┃ homeworld ┃ species ┃ films  ┃ vehicles ┃ starships ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━┩
│ string            │ int64  │ float64 │ string     │ string      │ string    │ float64    │ string │ string    │ string    │ string  │ string │ string   │ string    │
├───────────────────┼────────┼─────────┼────────────┼─────────────┼───────────┼────────────┼────────┼───────────┼───────────┼─────────┼────────┼──────────┼───────────┤
│ C-3PO             │    167 │    75.0 │ NULL       │ gold        │ yellow    │      112.0 │ none   │ masculine │ Tatooine  │ Droid   │ NULL   │ NULL     │ NULL      │
│ R2-D2             │     96 │    32.0 │ NULL       │ white, blue │ red       │       33.0 │ none   │ masculine │ Naboo     │ Droid   │ NULL   │ NULL     │ NULL      │
│ R5-D4             │     97 │    32.0 │ NULL       │ white, red  │ red       │        nan │ none   │ masculine │ Tatooine  │ Droid   │ NULL   │ NULL     │ NULL      │
│ IG-88             │    200 │   140.0 │ none       │ metal       │ red       │       15.0 │ none   │ masculine │ NULL      │ Droid   │ NULL   │ NULL     │ NULL      │
│ R4-P17            │     96 │     nan │ none       │ silver, red │ red, blue │        nan │ none   │ feminine  │ NULL      │ Droid   │ NULL   │ NULL     │ NULL      │
│ BB8               │   NULL │     nan │ none       │ none        │ black     │        nan │ none   │ masculine │ NULL      │ Droid   │ NULL   │ NULL     │ NULL      │
│ Leia Organa       │    150 │    49.0 │ brown      │ light       │ brown     │       19.0 │ female │ feminine  │ Alderaan  │ Human   │ NULL   │ NULL     │ NULL      │
│ Biggs Darklighter │    183 │    84.0 │ black      │ light       │ brown     │       24.0 │ male   │ masculine │ Tatooine  │ Human   │ NULL   │ NULL     │ NULL      │
│ Cordé             │    157 │     nan │ brown      │ light       │ brown     │        nan │ female │ feminine  │ Naboo     │ Human   │ NULL   │ NULL     │ NULL      │
│ Dormé             │    165 │     nan │ brown      │ light       │ brown     │        nan │ female │ feminine  │ Naboo     │ Human   │ NULL   │ NULL     │ NULL      │
│ …                 │      … │       … │ …          │ …           │ …         │          … │ …      │ …         │ …         │ …       │ …      │ …        │ …         │
└───────────────────┴────────┴─────────┴────────────┴─────────────┴───────────┴────────────┴────────┴───────────┴───────────┴─────────┴────────┴──────────┴───────────┘

### Sorting your data with order_by()

To sort a column, dplyr has the verb `arrange`. For instance, to sort the column `height` using dplyr:

```r
starwars |>
   arrange(height)
```

In Ibis:

In [ ]:
starwars.order_by(_.height)

┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┓
┃ name                  ┃ height ┃ mass    ┃ hair_color ┃ skin_color ┃ eye_color ┃ birth_year ┃ sex    ┃ gender    ┃ homeworld   ┃ species        ┃ films  ┃ vehicles ┃ starships ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━┩
│ string                │ int64  │ float64 │ string     │ string     │ string    │ float64    │ string │ string    │ string      │ string         │ string │ string   │ string    │
├───────────────────────┼────────┼─────────┼────────────┼────────────┼───────────┼────────────┼────────┼───────────┼─────────────┼────────────────┼────────┼──────────┼───────────┤
│ Arvel Crynyd          │   NULL │     nan │ brown      │ fair       │ brown     │        nan │ male   │ masculine │ NULL        │ Human          │ NULL   │ NULL     │ NULL      │
│ Finn                  │   NULL │     nan │ black      │ dark       │ dark      │        nan │ male   │ masculine │ NULL        │ Human          │ NULL   │ NULL     │ NULL      │
│ Rey                   │   NULL │     nan │ brown      │ light      │ hazel     │        nan │ female │ feminine  │ NULL        │ Human          │ NULL   │ NULL     │ NULL      │
│ Poe Dameron           │   NULL │     nan │ brown      │ light      │ brown     │        nan │ male   │ masculine │ NULL        │ Human          │ NULL   │ NULL     │ NULL      │
│ BB8                   │   NULL │     nan │ none       │ none       │ black     │        nan │ none   │ masculine │ NULL        │ Droid          │ NULL   │ NULL     │ NULL      │
│ Captain Phasma        │   NULL │     nan │ unknown    │ unknown    │ unknown   │        nan │ NULL   │ NULL      │ NULL        │ NULL           │ NULL   │ NULL     │ NULL      │
│ Yoda                  │     66 │    17.0 │ white      │ green      │ brown     │      896.0 │ male   │ masculine │ NULL        │ Yoda's species │ NULL   │ NULL     │ NULL      │
│ Ratts Tyerell         │     79 │    15.0 │ none       │ grey, blue │ unknown   │        nan │ male   │ masculine │ Aleen Minor │ Aleena         │ NULL   │ NULL     │ NULL      │
│ Wicket Systri Warrick │     88 │    20.0 │ brown      │ brown      │ brown     │        8.0 │ male   │ masculine │ Endor       │ Ewok           │ NULL   │ NULL     │ NULL      │
│ Dud Bolt              │     94 │    45.0 │ none       │ blue, grey │ yellow    │        nan │ male   │ masculine │ Vulpter     │ Vulptereen     │ NULL   │ NULL     │ NULL      │
│ …                     │      … │       … │ …          │ …          │ …         │          … │ …      │ …         │ …           │ …              │ …      │ …        │ …         │
└───────────────────────┴────────┴─────────┴────────────┴────────────┴───────────┴────────────┴────────┴───────────┴─────────────┴────────────────┴────────┴──────────┴───────────┘

You might notice that while dplyr puts missing values at the end, Ibis places them at the top.

If you want to order using multiple variables, you can pass them as a list:

In [ ]:
starwars.order_by([_.height, _.mass])

┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┓
┃ name                  ┃ height ┃ mass    ┃ hair_color ┃ skin_color ┃ eye_color ┃ birth_year ┃ sex    ┃ gender    ┃ homeworld   ┃ species        ┃ films  ┃ vehicles ┃ starships ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━┩
│ string                │ int64  │ float64 │ string     │ string     │ string    │ float64    │ string │ string    │ string      │ string         │ string │ string   │ string    │
├───────────────────────┼────────┼─────────┼────────────┼────────────┼───────────┼────────────┼────────┼───────────┼─────────────┼────────────────┼────────┼──────────┼───────────┤
│ Arvel Crynyd          │   NULL │     nan │ brown      │ fair       │ brown     │        nan │ male   │ masculine │ NULL        │ Human          │ NULL   │ NULL     │ NULL      │
│ Finn                  │   NULL │     nan │ black      │ dark       │ dark      │        nan │ male   │ masculine │ NULL        │ Human          │ NULL   │ NULL     │ NULL      │
│ Rey                   │   NULL │     nan │ brown      │ light      │ hazel     │        nan │ female │ feminine  │ NULL        │ Human          │ NULL   │ NULL     │ NULL      │
│ Poe Dameron           │   NULL │     nan │ brown      │ light      │ brown     │        nan │ male   │ masculine │ NULL        │ Human          │ NULL   │ NULL     │ NULL      │
│ BB8                   │   NULL │     nan │ none       │ none       │ black     │        nan │ none   │ masculine │ NULL        │ Droid          │ NULL   │ NULL     │ NULL      │
│ Captain Phasma        │   NULL │     nan │ unknown    │ unknown    │ unknown   │        nan │ NULL   │ NULL      │ NULL        │ NULL           │ NULL   │ NULL     │ NULL      │
│ Yoda                  │     66 │    17.0 │ white      │ green      │ brown     │      896.0 │ male   │ masculine │ NULL        │ Yoda's species │ NULL   │ NULL     │ NULL      │
│ Ratts Tyerell         │     79 │    15.0 │ none       │ grey, blue │ unknown   │        nan │ male   │ masculine │ Aleen Minor │ Aleena         │ NULL   │ NULL     │ NULL      │
│ Wicket Systri Warrick │     88 │    20.0 │ brown      │ brown      │ brown     │        8.0 │ male   │ masculine │ Endor       │ Ewok           │ NULL   │ NULL     │ NULL      │
│ Dud Bolt              │     94 │    45.0 │ none       │ blue, grey │ yellow    │        nan │ male   │ masculine │ Vulpter     │ Vulptereen     │ NULL   │ NULL     │ NULL      │
│ …                     │      … │       … │ …          │ …          │ …         │          … │ …      │ …         │ …           │ …              │ …      │ …        │ …         │
└───────────────────────┴────────┴─────────┴────────────┴────────────┴───────────┴────────────┴────────┴───────────┴─────────────┴────────────────┴────────┴──────────┴───────────┘

To order a column in descending order, there are two ways to it. Note that missing values remain at the top.

In [ ]:
starwars.order_by(_.height.desc()) # or: starwars.order_by(ibis.desc("height"))

┏━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┓
┃ name           ┃ height ┃ mass    ┃ hair_color ┃ skin_color ┃ eye_color ┃ birth_year ┃ sex    ┃ gender    ┃ homeworld ┃ species  ┃ films  ┃ vehicles ┃ starships ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━┩
│ string         │ int64  │ float64 │ string     │ string     │ string    │ float64    │ string │ string    │ string    │ string   │ string │ string   │ string    │
├────────────────┼────────┼─────────┼────────────┼────────────┼───────────┼────────────┼────────┼───────────┼───────────┼──────────┼────────┼──────────┼───────────┤
│ Arvel Crynyd   │   NULL │     nan │ brown      │ fair       │ brown     │        nan │ male   │ masculine │ NULL      │ Human    │ NULL   │ NULL     │ NULL      │
│ Finn           │   NULL │     nan │ black      │ dark       │ dark      │        nan │ male   │ masculine │ NULL      │ Human    │ NULL   │ NULL     │ NULL      │
│ Rey            │   NULL │     nan │ brown      │ light      │ hazel     │        nan │ female │ feminine  │ NULL      │ Human    │ NULL   │ NULL     │ NULL      │
│ Poe Dameron    │   NULL │     nan │ brown      │ light      │ brown     │        nan │ male   │ masculine │ NULL      │ Human    │ NULL   │ NULL     │ NULL      │
│ BB8            │   NULL │     nan │ none       │ none       │ black     │        nan │ none   │ masculine │ NULL      │ Droid    │ NULL   │ NULL     │ NULL      │
│ Captain Phasma │   NULL │     nan │ unknown    │ unknown    │ unknown   │        nan │ NULL   │ NULL      │ NULL      │ NULL     │ NULL   │ NULL     │ NULL      │
│ Yarael Poof    │    264 │     nan │ none       │ white      │ yellow    │        nan │ male   │ masculine │ Quermia   │ Quermian │ NULL   │ NULL     │ NULL      │
│ Tarfful        │    234 │   136.0 │ brown      │ brown      │ blue      │        nan │ male   │ masculine │ Kashyyyk  │ Wookiee  │ NULL   │ NULL     │ NULL      │
│ Lama Su        │    229 │    88.0 │ none       │ grey       │ black     │        nan │ male   │ masculine │ Kamino    │ Kaminoan │ NULL   │ NULL     │ NULL      │
│ Chewbacca      │    228 │   112.0 │ brown      │ unknown    │ blue      │      200.0 │ male   │ masculine │ Kashyyyk  │ Wookiee  │ NULL   │ NULL     │ NULL      │
│ …              │      … │       … │ …          │ …          │ …         │          … │ …      │ …         │ …         │ …        │ …      │ …        │ …         │
└────────────────┴────────┴─────────┴────────────┴────────────┴───────────┴────────────┴────────┴───────────┴───────────┴──────────┴────────┴──────────┴───────────┘

### Selecting rows by their index

dplyr provides several functions in the `slice` family to select some rows from the dataset. They are not directly implemented in Ibis but can be emulated with other functions.

For instance, in dplyr, you can use `slice` to select rows 5 to 10:

```r
starwars |>
   slice(5:10)
```

In Ibis, you can use `limit` and specify an offset:

In [ ]:
starwars.limit(5, offset = 4)

┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┓
┃ name               ┃ height ┃ mass    ┃ hair_color  ┃ skin_color ┃ eye_color ┃ birth_year ┃ sex    ┃ gender    ┃ homeworld ┃ species ┃ films  ┃ vehicles ┃ starships ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━┩
│ string             │ int64  │ float64 │ string      │ string     │ string    │ float64    │ string │ string    │ string    │ string  │ string │ string   │ string    │
├────────────────────┼────────┼─────────┼─────────────┼────────────┼───────────┼────────────┼────────┼───────────┼───────────┼─────────┼────────┼──────────┼───────────┤
│ Leia Organa        │    150 │    49.0 │ brown       │ light      │ brown     │       19.0 │ female │ feminine  │ Alderaan  │ Human   │ NULL   │ NULL     │ NULL      │
│ Owen Lars          │    178 │   120.0 │ brown, grey │ light      │ blue      │       52.0 │ male   │ masculine │ Tatooine  │ Human   │ NULL   │ NULL     │ NULL      │
│ Beru Whitesun lars │    165 │    75.0 │ brown       │ light      │ blue      │       47.0 │ female │ feminine  │ Tatooine  │ Human   │ NULL   │ NULL     │ NULL      │
│ R5-D4              │     97 │    32.0 │ NULL        │ white, red │ red       │        nan │ none   │ masculine │ Tatooine  │ Droid   │ NULL   │ NULL     │ NULL      │
│ Biggs Darklighter  │    183 │    84.0 │ black       │ light      │ brown     │       24.0 │ male   │ masculine │ Tatooine  │ Human   │ NULL   │ NULL     │ NULL      │
└────────────────────┴────────┴─────────┴─────────────┴────────────┴───────────┴────────────┴────────┴───────────┴───────────┴─────────┴────────┴──────────┴───────────┘

TODO: `slice_sample` is not implemented. (add example on how to do it?)
TODO: `slice_max` and `slice_min` are not implemented but similar results can be obtained combining `order_by` and `limit`:

In [ ]:
(
    starwars
        .filter(_.height.notnull())
        .order_by(_.height.desc())
        .limit(3)
)

┏━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┓
┃ name        ┃ height ┃ mass    ┃ hair_color ┃ skin_color ┃ eye_color ┃ birth_year ┃ sex    ┃ gender    ┃ homeworld ┃ species  ┃ films  ┃ vehicles ┃ starships ┃
┡━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━┩
│ string      │ int64  │ float64 │ string     │ string     │ string    │ float64    │ string │ string    │ string    │ string   │ string │ string   │ string    │
├─────────────┼────────┼─────────┼────────────┼────────────┼───────────┼────────────┼────────┼───────────┼───────────┼──────────┼────────┼──────────┼───────────┤
│ Yarael Poof │    264 │     nan │ none       │ white      │ yellow    │        nan │ male   │ masculine │ Quermia   │ Quermian │ NULL   │ NULL     │ NULL      │
│ Tarfful     │    234 │   136.0 │ brown      │ brown      │ blue      │        nan │ male   │ masculine │ Kashyyyk  │ Wookiee  │ NULL   │ NULL     │ NULL      │
│ Lama Su     │    229 │    88.0 │ none       │ grey       │ black     │        nan │ male   │ masculine │ Kamino    │ Kaminoan │ NULL   │ NULL     │ NULL      │
└─────────────┴────────┴─────────┴────────────┴────────────┴───────────┴────────────┴────────┴───────────┴───────────┴──────────┴────────┴──────────┴───────────┘

### Selecting columns with select()

Ibis, like dplyr, has a `select` method to select or exclude columns:

With dplyr:

```r
starwars |> 
    select(hair_color)
```

In Ibis:

In [ ]:
starwars.select(_.hair_color)

┏━━━━━━━━━━━━━━━┓
┃ hair_color    ┃
┡━━━━━━━━━━━━━━━┩
│ string        │
├───────────────┤
│ blond         │
│ NULL          │
│ NULL          │
│ none          │
│ brown         │
│ brown, grey   │
│ brown         │
│ NULL          │
│ black         │
│ auburn, white │
│ …             │
└───────────────┘

dplyr also allows selecting more than one column at a time:

```r
starwars |>
    select(hair_color, skin_color, eye_color)
```

In Ibis, you can pass a list of column names:

In [ ]:
starwars.select(["hair_color", "skin_color", "eye_color"])

┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ hair_color    ┃ skin_color  ┃ eye_color ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ string        │ string      │ string    │
├───────────────┼─────────────┼───────────┤
│ blond         │ fair        │ blue      │
│ NULL          │ gold        │ yellow    │
│ NULL          │ white, blue │ red       │
│ none          │ white       │ yellow    │
│ brown         │ light       │ brown     │
│ brown, grey   │ light       │ blue      │
│ brown         │ light       │ blue      │
│ NULL          │ white, red  │ red       │
│ black         │ light       │ brown     │
│ auburn, white │ fair        │ blue-gray │
│ …             │ …           │ …         │
└───────────────┴─────────────┴───────────┘

To select columns by name based on a condition, dplyr has helpers such as:

- starts_with(): Starts with a prefix.
- ends_with(): Ends with a suffix.
- contains(): Contains a literal string.

These are available in Ibis as well, with slightly different names:

In [4]:
starwars.select(s.startswith("h"))

NameError: name 'starwars' is not defined

In [ ]:
starwars.select(s.endswith("color"))

┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ hair_color    ┃ skin_color  ┃ eye_color ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ string        │ string      │ string    │
├───────────────┼─────────────┼───────────┤
│ blond         │ fair        │ blue      │
│ NULL          │ gold        │ yellow    │
│ NULL          │ white, blue │ red       │
│ none          │ white       │ yellow    │
│ brown         │ light       │ brown     │
│ brown, grey   │ light       │ blue      │
│ brown         │ light       │ blue      │
│ NULL          │ white, red  │ red       │
│ black         │ light       │ brown     │
│ auburn, white │ fair        │ blue-gray │
│ …             │ …           │ …         │
└───────────────┴─────────────┴───────────┘

In [ ]:
starwars.select(s.contains("world"))

┏━━━━━━━━━━━┓
┃ homeworld ┃
┡━━━━━━━━━━━┩
│ string    │
├───────────┤
│ Tatooine  │
│ Tatooine  │
│ Naboo     │
│ Tatooine  │
│ Alderaan  │
│ Tatooine  │
│ Tatooine  │
│ Tatooine  │
│ Tatooine  │
│ Stewjon   │
│ …         │
└───────────┘

### Renaming columns with relabel()

Ibis allows you to rename columns using `relabel()` which provides similar functionality to `rename()` in dplyr.

In dplyr:

```r
starwars |> 
    rename("homeworld"="home_world")
```

In Ibis, use `relabel` and pass a `dict` or name mappings:

In [ ]:
starwars.relabel({"homeworld": "home_world"})

┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┓
┃ name               ┃ height ┃ mass    ┃ hair_color    ┃ skin_color  ┃ eye_color ┃ birth_year ┃ sex    ┃ gender    ┃ home_world ┃ species ┃ films  ┃ vehicles ┃ starships ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━┩
│ string             │ int64  │ float64 │ string        │ string      │ string    │ float64    │ string │ string    │ string     │ string  │ string │ string   │ string    │
├────────────────────┼────────┼─────────┼───────────────┼─────────────┼───────────┼────────────┼────────┼───────────┼────────────┼─────────┼────────┼──────────┼───────────┤
│ Luke Skywalker     │    172 │    77.0 │ blond         │ fair        │ blue      │       19.0 │ male   │ masculine │ Tatooine   │ Human   │ NULL   │ NULL     │ NULL      │
│ C-3PO              │    167 │    75.0 │ NULL          │ gold        │ yellow    │      112.0 │ none   │ masculine │ Tatooine   │ Droid   │ NULL   │ NULL     │ NULL      │
│ R2-D2              │     96 │    32.0 │ NULL          │ white, blue │ red       │       33.0 │ none   │ masculine │ Naboo      │ Droid   │ NULL   │ NULL     │ NULL      │
│ Darth Vader        │    202 │   136.0 │ none          │ white       │ yellow    │       41.9 │ male   │ masculine │ Tatooine   │ Human   │ NULL   │ NULL     │ NULL      │
│ Leia Organa        │    150 │    49.0 │ brown         │ light       │ brown     │       19.0 │ female │ feminine  │ Alderaan   │ Human   │ NULL   │ NULL     │ NULL      │
│ Owen Lars          │    178 │   120.0 │ brown, grey   │ light       │ blue      │       52.0 │ male   │ masculine │ Tatooine   │ Human   │ NULL   │ NULL     │ NULL      │
│ Beru Whitesun lars │    165 │    75.0 │ brown         │ light       │ blue      │       47.0 │ female │ feminine  │ Tatooine   │ Human   │ NULL   │ NULL     │ NULL      │
│ R5-D4              │     97 │    32.0 │ NULL          │ white, red  │ red       │        nan │ none   │ masculine │ Tatooine   │ Droid   │ NULL   │ NULL     │ NULL      │
│ Biggs Darklighter  │    183 │    84.0 │ black         │ light       │ brown     │       24.0 │ male   │ masculine │ Tatooine   │ Human   │ NULL   │ NULL     │ NULL      │
│ Obi-Wan Kenobi     │    182 │    77.0 │ auburn, white │ fair        │ blue-gray │       57.0 │ male   │ masculine │ Stewjon    │ Human   │ NULL   │ NULL     │ NULL      │
│ …                  │      … │       … │ …             │ …           │ …         │          … │ …      │ …         │ …          │ …       │ …      │ …        │ …         │
└────────────────────┴────────┴─────────┴───────────────┴─────────────┴───────────┴────────────┴────────┴───────────┴────────────┴─────────┴────────┴──────────┴───────────┘

### Add new columns with mutate()

Ibis, like dplyr, uses the `mutate` verb to add columns.

In dplyr,

```r
starwars |>
    mutate(height_m = height / 100) |>
    select(name, height_m)
```

In Ibis:

In [ ]:
(
    starwars
        .mutate(height_m = _.height / 100)
        .select("name", "height_m")
)

┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ name               ┃ height_m ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━┩
│ string             │ float64  │
├────────────────────┼──────────┤
│ Luke Skywalker     │     1.72 │
│ C-3PO              │     1.67 │
│ R2-D2              │     0.96 │
│ Darth Vader        │     2.02 │
│ Leia Organa        │     1.50 │
│ Owen Lars          │     1.78 │
│ Beru Whitesun lars │     1.65 │
│ R5-D4              │     0.97 │
│ Biggs Darklighter  │     1.83 │
│ Obi-Wan Kenobi     │     1.82 │
│ …                  │        … │
└────────────────────┴──────────┘

A big difference between dplyr's `mutate` and Ibis' is that, in Ibis, you have to chain separate `mutate` calls together you reference newly-created columns in the same `mutate` whereas in dplyr, you can put them all in the same call. This makes Ibis' `mutate` more similar to `transform` in dplyr.

In dplyr,

```r
starwars %>%
  mutate(
    height_m = height / 100,
    BMI = mass / (height_m^2)
  ) %>%
  select(BMI, everything())
  ```

In Ibis, for `BMI` to reference `height_m`, it needs to be in a separate `mutate` call:

In [ ]:
(starwars
    .mutate(
        height_m = _.height / 100
    )
    .mutate(        
        BMI = _.mass / (_.height_m**2)
    )
    .select("BMI", ~s.matches("BMI"))
)

┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┓
┃ BMI       ┃ name               ┃ height ┃ mass    ┃ hair_color    ┃ skin_color  ┃ eye_color ┃ birth_year ┃ sex    ┃ gender    ┃ homeworld ┃ species ┃ films  ┃ vehicles ┃ starships ┃ height_m ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━┩
│ float64   │ string             │ int64  │ float64 │ string        │ string      │ string    │ float64    │ string │ string    │ string    │ string  │ string │ string   │ string    │ float64  │
├───────────┼────────────────────┼────────┼─────────┼───────────────┼─────────────┼───────────┼────────────┼────────┼───────────┼───────────┼─────────┼────────┼──────────┼───────────┼──────────┤
│ 26.027582 │ Luke Skywalker     │    172 │    77.0 │ blond         │ fair        │ blue      │       19.0 │ male   │ masculine │ Tatooine  │ Human   │ NULL   │ NULL     │ NULL      │     1.72 │
│ 26.892323 │ C-3PO              │    167 │    75.0 │ NULL          │ gold        │ yellow    │      112.0 │ none   │ masculine │ Tatooine  │ Droid   │ NULL   │ NULL     │ NULL      │     1.67 │
│ 34.722222 │ R2-D2              │     96 │    32.0 │ NULL          │ white, blue │ red       │       33.0 │ none   │ masculine │ Naboo     │ Droid   │ NULL   │ NULL     │ NULL      │     0.96 │
│ 33.330066 │ Darth Vader        │    202 │   136.0 │ none          │ white       │ yellow    │       41.9 │ male   │ masculine │ Tatooine  │ Human   │ NULL   │ NULL     │ NULL      │     2.02 │
│ 21.777778 │ Leia Organa        │    150 │    49.0 │ brown         │ light       │ brown     │       19.0 │ female │ feminine  │ Alderaan  │ Human   │ NULL   │ NULL     │ NULL      │     1.50 │
│ 37.874006 │ Owen Lars          │    178 │   120.0 │ brown, grey   │ light       │ blue      │       52.0 │ male   │ masculine │ Tatooine  │ Human   │ NULL   │ NULL     │ NULL      │     1.78 │
│ 27.548209 │ Beru Whitesun lars │    165 │    75.0 │ brown         │ light       │ blue      │       47.0 │ female │ feminine  │ Tatooine  │ Human   │ NULL   │ NULL     │ NULL      │     1.65 │
│ 34.009990 │ R5-D4              │     97 │    32.0 │ NULL          │ white, red  │ red       │        nan │ none   │ masculine │ Tatooine  │ Droid   │ NULL   │ NULL     │ NULL      │     0.97 │
│ 25.082863 │ Biggs Darklighter  │    183 │    84.0 │ black         │ light       │ brown     │       24.0 │ male   │ masculine │ Tatooine  │ Human   │ NULL   │ NULL     │ NULL      │     1.83 │
│ 23.245985 │ Obi-Wan Kenobi     │    182 │    77.0 │ auburn, white │ fair        │ blue-gray │       57.0 │ male   │ masculine │ Stewjon   │ Human   │ NULL   │ NULL     │ NULL      │     1.82 │
│         … │ …                  │      … │       … │ …             │ …           │ …         │          … │ …      │ …         │ …         │ …       │ …      │ …        │ …         │        … │
└───────────┴────────────────────┴────────┴─────────┴───────────────┴─────────────┴───────────┴────────────┴────────┴───────────┴───────────┴─────────┴────────┴──────────┴───────────┴──────────┘

### Summarize values with aggregate()

To summarize tables, dplyr has the verb `summarize`/`summarize`:

In dplyr:

```r
starwars %>% 
    summarise(height = mean(height, na.rm = TRUE))
```

In Ibis, the corresponding verb is `aggregate`:

In [ ]:
starwars.aggregate(height = _.height.mean())

┏━━━━━━━━━━━━┓
┃ height     ┃
┡━━━━━━━━━━━━┩
│ float64    │
├────────────┤
│ 174.358025 │
└────────────┘

### Joins

TODO

### Pivot

dplyr users are likely to be familiar with the `pivot_wider` and `pivot_longer` functions from the [tidyr](https://tidyr.tidyverse.org) package which convert tables between wide and long formats, respectively.

In dplyr+tidyr:

```r
starwars |> 
    select(name:mass) |> 
    pivot_longer(height:mass, names_to="property", values_to="value")
```

In Ibis:

In [ ]:
starwars_longer = (
    starwars
        .pivot_longer(s.matches("color"))
        .select(["name", "property", "value"])
)

starwars_longer

NameError: name 'starwars' is not defined

And the reverse:

In [5]:
# TODO
# (
#     starwars_longer.pivot_wider()
# )